# 03: Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/03_visualization.ipynb)

Visualizing snowpack data is crucial for understanding patterns and communicating results. This notebook shows you how to create effective plots with xsnow data.

## What You'll Learn

- Plotting snow profiles (depth vs properties)
- Creating time series plots
- Multi-panel figures
- Customizing plots for presentations
- Temperature and density profiles


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [1]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.3 which is incompatible.
xsnow 0.0.1 requires numpy>=2.3.2, but you have numpy 1.26.4 which is incompatible.
dask-expr 1.1.13 requires dask==2024.8.2, but you have dask 2025.11.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.4 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.3 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.3.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, 

In [1]:
import xsnow
import matplotlib.pyplot as plt
import numpy as np

# Load sample data
ds = xsnow.single_profile_timeseries()
ds

%matplotlib inline



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed

## Part 1: Plotting a Single Snow Profile

A snow profile shows properties (like density, temperature) as a function of depth. This is the classic "snow pit" visualization.


In [ ]:
# Select a single profile (first location, first time)
profile = ds.isel(location=0, time=0, slope=0, realization=0)

# Get depth (z coordinate) - convert to positive depth for plotting
depth = -profile.coords['z'].values  # Make positive (depth from surface)
temp = profile['temperature'].values

# Plot temperature profile
fig, ax = plt.subplots(figsize=(6, 8))
ax.plot(temp, depth, 'r-', linewidth=2, label='Temperature')
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)  # 0°C reference

ax.set_xlabel('Temperature (°C)', fontsize=12)
ax.set_ylabel('Depth from surface (m)', fontsize=12)
ax.set_title('Temperature Profile', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_yaxis()  # Surface at top

plt.tight_layout()
plt.show()


### 🔧 Try This: Tweak, Twiddle, and Frob!

**Tweak, Twiddle, and Frob Pattern**: Experiment with the plot above! Try modifying these parameters to see how they affect the visualization:

- Change the line color: `'r-'` to `'b-'` (blue), `'g-'` (green), or `'m-'` (magenta)
- Modify `linewidth=2` to `linewidth=3` or `linewidth=1` for thicker/thinner lines
- Change `figsize=(6, 8)` to `figsize=(10, 12)` for a larger plot or `figsize=(4, 6)` for smaller
- Try different line styles: `'r-'` (solid), `'r--'` (dashed), `'r:'` (dotted), `'r-.'` (dash-dot)
- Add markers: change `'r-'` to `'ro'` (red circles) or `'r^'` (red triangles)


### Density Profile

Let's also plot density:


**Now You Try**: 
- Plot a density profile for a different time step (hint: change `time=0` to `time=5` or another index)
- Plot a profile for a different location (hint: change `location=0`)
- Create a plot showing both temperature and density on the same axes (hint: use `ax.plot()` twice with different colors)


In [ ]:
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values
density = profile['density'].values

fig, ax = plt.subplots(figsize=(6, 8))
ax.plot(density, depth, 'b-', linewidth=2, label='Density')

ax.set_xlabel('Density (kg/m³)', fontsize=12)
ax.set_ylabel('Depth from surface (m)', fontsize=12)
ax.set_title('Density Profile', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_yaxis()

plt.tight_layout()
plt.show()


### 🔧 Try This: Experiment with Density Plot

**Tweak, Twiddle, and Frob**: Modify the density plot above:
- Change the color from `'b-'` (blue) to `'g-'` (green) or `'c-'` (cyan)
- Adjust the figure size: try `figsize=(8, 10)` or `figsize=(5, 7)`
- Add a vertical reference line at density = 300 kg/m³ (hint: use `ax.axvline(x=300, ...)`)
- Try plotting with markers: change `'b-'` to `'bo'` or `'bs'` (squares)


## Part 2: Multi-Panel Profile Plot

Combine multiple properties in one figure (like a real snow pit diagram):


**Now You Try**:
- Create a multi-panel plot with 4 subplots instead of 3 (hint: change `(1, 3, ...)` to `(2, 2, ...)`)
- Add a fourth variable to the plot (try grain type, grain size, or another available variable)
- Change the layout from horizontal `(1, 3, ...)` to vertical `(3, 1, ...)`


In [ ]:
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values

# Create figure with multiple subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)

# Temperature
axes[0].plot(profile['temperature'].values, depth, 'r-', linewidth=2)
axes[0].axvline(x=0, color='k', linestyle='--', alpha=0.3)
axes[0].set_xlabel('Temperature (°C)', fontsize=11)
axes[0].set_title('Temperature', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].invert_yaxis()

# Density
axes[1].plot(profile['density'].values, depth, 'b-', linewidth=2)
axes[1].set_xlabel('Density (kg/m³)', fontsize=11)
axes[1].set_title('Density', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].invert_yaxis()

# Third plot - use another variable if available, or show density gradient
if 'grain_size' in profile.data_vars:
    axes[2].plot(profile['grain_size'].values, depth, 'g-', linewidth=2)
    axes[2].set_xlabel('Grain Size (mm)', fontsize=11)
    axes[2].set_title('Grain Size', fontsize=12, fontweight='bold')
else:
    # Show density gradient instead
    density_grad = profile['density'].diff(dim='layer')
    axes[2].plot(density_grad.values, depth[:-1], 'g-', linewidth=2)
    axes[2].set_xlabel('Density Gradient', fontsize=11)
    axes[2].set_title('Density Gradient', fontsize=12, fontweight='bold')

axes[2].grid(True, alpha=0.3)
axes[2].invert_yaxis()

# Set shared y-axis label
axes[0].set_ylabel('Depth from surface (m)', fontsize=12)

plt.tight_layout()
plt.show()


**Now You Try**:
- Plot snow height for a different location
- Plot a different variable over time (try surface temperature or mean density)
- Add a second y-axis to show two variables on the same plot (hint: use `ax2 = ax.twinx()`)


## Part 3: Time Series Plots

Plot how snowpack properties change over time:


**Now You Try**:
- Plot temperature for a different layer (not just layer 0)
- Plot temperature at a specific depth (e.g., 50 cm below surface)
- Create a plot showing both surface and bottom layer temperatures


In [ ]:
# Get time series of snow height
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times, hs_series.values, 'b-', linewidth=2, label='Snow Height (HS)')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height Time Series', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Rotate x-axis labels for readability
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 🔧 Try This: Experiment with Heatmaps

**Tweak, Twiddle, and Frob**: Modify the heatmap above:
- Change the colormap: `'RdYlBu_r'` to `'viridis'`, `'plasma'`, `'coolwarm'`, or `'seismic'`
- Modify the figure size: try `figsize=(20, 10)` for a wider plot or `figsize=(10, 6)` for smaller
- Adjust the colorbar: try `cbar.set_label('Temperature (°C)', fontsize=14)` for larger text
- Change the shading: try `shading='gouraud'` instead of `'auto'` for smoother gradients


**Now You Try**:
- Create a heatmap for density instead of temperature
- Create a heatmap for a different location
- Add contour lines to the heatmap (hint: use `ax.contour()`)


### Surface Temperature Time Series

Track surface layer temperature over time:


**Now You Try**:
- Compare a different variable (e.g., temperature or density) across locations
- Create separate subplots for each location instead of overlaying them
- Add error bars or confidence intervals if you have multiple realizations


In [ ]:
# Get surface temperature (layer 0) over time
surface_temp = ds['temperature'].isel(location=0, layer=0, slope=0, realization=0)
times = ds.coords['time'].values

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times, surface_temp.values, 'r-', linewidth=2, label='Surface Temperature')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Freezing Point')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)
ax.set_title('Surface Temperature Time Series', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 🔧 Try This: Customize Your Plot

**Tweak, Twiddle, and Frob**: Experiment with the customization options:
- Change marker style: `'o'` to `'s'` (square), `'^'` (triangle), `'d'` (diamond), or `'*'` (star)
- Modify marker size: `markersize=6` to `markersize=10` or `markersize=4`
- Adjust line width: `linewidth=3` to `linewidth=5` for thicker lines
- Change transparency: `alpha=0.8` to `alpha=0.5` (more transparent) or `alpha=1.0` (opaque)
- Try different font sizes: change `fontsize=14` to `fontsize=16` or `fontsize=12`
- Experiment with colors: try `'#FF5733'` (custom hex color) or named colors like `'darkblue'`, `'crimson'`


**Now You Try**:
- Save your plot to a file (hint: use `plt.savefig('my_plot.png', dpi=300)`)
- Create a plot with a different style (try `plt.style.use('seaborn-v0_8')` or `'ggplot'`)
- Add annotations pointing to specific features in your plot (hint: use `ax.annotate()`)


## Part 4: Heatmap (Depth-Time)

A heatmap shows how properties change with both depth and time - very useful for seeing evolution:


In [ ]:
# Select one location
temp_data = ds['temperature'].isel(location=0, slope=0, realization=0)

# Get coordinates
times = ds.coords['time'].values
# Get z coordinates (depth) - use first time step as reference
z_coords = ds.coords['z'].isel(location=0, time=0, slope=0, realization=0).values
depth = -z_coords  # Convert to positive depth

# Filter out NaN values from depth (layers that don't exist)
valid_mask = ~np.isnan(depth)
depth_clean = depth[valid_mask]

# Prepare data - temp_data shape is (time, layer)
# We need (layer, time) for plotting
temp_plot = temp_data.values.T

# Filter temperature data to match valid depth layers
temp_plot_clean = temp_plot[valid_mask, :]

# Create heatmap
fig, ax = plt.subplots(figsize=(14, 8))

# Use pcolormesh for better results with this data structure
# Ensure times doesn't have NaN values
if np.any(np.isnan(times)):
    # If times has NaN, filter them out
    times_valid = times[~np.isnan(times)]
    temp_plot_clean = temp_plot_clean[:, ~np.isnan(times)]
else:
    times_valid = times

im = ax.pcolormesh(times_valid, depth_clean, temp_plot_clean, cmap='RdYlBu_r', shading='auto')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Temperature (°C)', fontsize=12)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Depth from surface (m)', fontsize=12)
ax.set_title('Temperature Evolution (Depth-Time)', fontsize=14, fontweight='bold')
ax.invert_yaxis()  # Surface at top

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Part 5: Comparing Multiple Locations

If you have data from multiple locations, compare them:


In [ ]:
# Compare multiple locations
times = ds.coords['time'].values
n_locations = ds.sizes.get('location', 1)

fig, ax = plt.subplots(figsize=(12, 6))

# Plot each location
for i in range(n_locations):
    location_name = ds.coords['location'].values[i]
    hs_series = ds['HS'].isel(location=i, slope=0, realization=0)
    ax.plot(times, hs_series.values, linewidth=2, label=f'Location {location_name}')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height Comparison', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Part 6: Customizing Plots

Make publication-quality figures:


In [ ]:
# Example of a highly customized plot
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values
temp = profile['temperature'].values

# Create figure with custom styling
fig, ax = plt.subplots(figsize=(8, 10))

# Plot with custom style
ax.plot(temp, depth, 'r-', linewidth=3, marker='o', markersize=6, 
        label='Temperature', alpha=0.8)
ax.axvline(x=0, color='k', linestyle='--', linewidth=1.5, alpha=0.5, label='Freezing')

# Customize axes
ax.set_xlabel('Temperature (°C)', fontsize=14, fontweight='bold')
ax.set_ylabel('Depth from surface (m)', fontsize=14, fontweight='bold')
ax.set_title('Snow Temperature Profile', fontsize=16, fontweight='bold', pad=20)

# Customize grid
ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
ax.set_axisbelow(True)

# Customize ticks
ax.tick_params(labelsize=11)

# Legend
ax.legend(fontsize=12, framealpha=0.9)

# Invert y-axis
ax.invert_yaxis()

# Add text annotation
ax.text(0.05, 0.95, 'Surface', transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


## Summary

✅ **What we learned:**

1. **Single profiles**: Plot temperature/density vs depth
2. **Multi-panel plots**: Combine multiple properties
3. **Time series**: Track changes over time
4. **Heatmaps**: Visualize depth-time evolution
5. **Comparisons**: Compare multiple locations
6. **Customization**: Make publication-quality figures

## Key Plotting Tips

- Always invert y-axis for depth plots (surface at top)
- Use `sharey=True` for multi-panel profile plots
- Rotate date labels for readability: `plt.xticks(rotation=45)`
- Save high-resolution figures: `plt.savefig('plot.png', dpi=300)`
- Use appropriate colormaps for heatmaps (e.g., 'RdYlBu_r' for temperature)

## Next Steps

Ready for advanced analysis? Move on to:
- **04_advanced_analysis.ipynb**: Stability indices, hazard calculations, and more

## Exercises

1. Create a temperature profile for a specific date
2. Plot snow height over the entire time period
3. Create a heatmap showing density evolution
4. Compare surface temperature between two different dates
5. Make a publication-quality multi-panel figure with 4 subplots
